In [ ]:

import logging
logging.basicConfig(level=logging.ERROR)  
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import sys
import seaborn as sns
sys.path.append('/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/')
from src.cmos_plotter.Plotter_Helper_KV import *
from src.utils.metadata_functions import load_metadata_as_dataframe
from src.utils.logger_functions import console
from src.cmos_plotter import Conduction_speed_plotter as csp
import spikeinterface.extractors as se
import pickle
from matplotlib import cm

In [ ]:
MAIN_PATH = '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Small_Network_Plasticity/3_Processed_Data/BCM_Test'
RECORDINGS_PATH = '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Small_Network_Plasticity/2_Raw_Data/Plasticity_Experiments/Raw_Data/BCM_Test/1821'
SORTER_PATH = os.path.join(MAIN_PATH, f'Sorters_old_version')
CONDUCTION_SPEEDS_PATH = os.path.join(MAIN_PATH, f'Results/Conduction_speeds')

if not os.path.exists(CONDUCTION_SPEEDS_PATH):
    os.makedirs(CONDUCTION_SPEEDS_PATH)

metadata = load_metadata_as_dataframe(file_path=MAIN_PATH)
metadata

In [ ]:
results_combined = [] 
for idx, filename in enumerate(metadata.Filename):
    chip_id = metadata.loc[idx, 'Chip_ID']
    div = metadata.loc[idx, 'DIV']
    cell_type = 'NGN2'
    area = metadata.loc[idx, 'Network_ID']
    
    try:
        print('Calculating conduction speeds for:', filename)
        recording = se.MaxwellRecordingExtractor(os.path.join(RECORDINGS_PATH, f'{filename}'))
        template_path = os.path.join(SORTER_PATH, f'Sorter_{filename}/wf_folder_curated/')
        
        # Get sampling rate
        sampling_rate = recording.get_sampling_frequency()
        # Get probe locations
        probe_locations = recording.get_channel_locations()
        # Extract templates
        templates = csp.load_and_extract_templates(template_path)
        # Analyze conduction speeds
        results = csp.analyze_conduction_speeds(templates, probe_locations, sampling_rate)
        
        if results is not None and len(results) > 0: 
            # Store recording-specific metadata for later use in visualization
            recording_metadata = {
                'chip_id': chip_id,
                'div': div,
                'cell_type': cell_type,
                'area': area,
                'sampling_rate': sampling_rate,
                'probe_locations': probe_locations,
                'templates': templates,
                'filename': filename
            }
            
            # Add metadata to each unit's results
            for result in results:
                result['chip_id'] = chip_id
                result['div'] = div
                result['cell_type'] = cell_type
                result['area'] = area
                result['filename'] = filename
                # Makes sure the speed key is consistent
                if 'speed' in result and 'speed_ms-1' not in result:
                    result['speed_ms-1'] = result.pop('speed')
                
                # Add to the combined results list
                results_combined.append(result)
            
            # Save the metadata separately for this recording
            metadata_dict = {
                'sampling_rate': sampling_rate,
                'probe_locations': probe_locations,
                'templates': templates
            }
            
            # Save individual recording metadata
            with open(os.path.join(CONDUCTION_SPEEDS_PATH, f'metadata_{filename}.pkl'), 'wb') as f:
                pickle.dump(metadata_dict, f)
                
        else:
            print(f'No conduction speeds calculated for {filename}')
            
    except Exception as e:
        print(f'Error calculating conduction speeds for {filename}: {e}')
        continue

# Save combined results
with open(os.path.join(CONDUCTION_SPEEDS_PATH, 'Conduction_speeds_all.pkl'), 'wb') as f:
    pickle.dump(results_combined, f)

In [ ]:
results_combined = pd.DataFrame(results_combined)
results_combined

In [ ]:
conduction_speeds = np.load(os.path.join(CONDUCTION_SPEEDS_PATH, 'Conduction_speeds_all.pkl'), allow_pickle=True)

In [ ]:

#_ = csp.visualize_unit_propagation_V2(CONDUCTION_SPEEDS_PATH, filename, results, templates, probe_locations, 
#                                sampling_rate)
#RUN THE UPPER FUNCTION FOR EACH FILENAME IN CONDUCTION SPEEDS
for i in range(len(conduction_speeds['results'])):
    filename = conduction_speeds['filename']
    results = conduction_speeds['results']
    templates = conduction_speeds['templates']
    probe_locations = conduction_speeds['probe_locations']
    sampling_rate = conduction_speeds[i]['sampling_rate']
    _ = csp.visualize_unit_propagation_V2(CONDUCTION_SPEEDS_PATH, filename, results, templates, probe_locations, 
                                sampling_rate)


In [ ]:
csp.visualize_propagation_by_recording(CONDUCTION_SPEEDS_PATH, CONDUCTION_SPEEDS_PATH)